# Assignment: SVD Preprocessing on MNIST with Logistic Regression

## Instructions:
In this assignment, you will apply **Singular Value Decomposition (SVD)** as a preprocessing step to the **MNIST dataset** and train a **logistic regression classifier**. You will compare the model performance and training time when using different levels of SVD for dimensionality reduction.

In this assignment, you will need to:
1. Load the MNIST dataset and normalize it.
2. Perform SVD and reduce the dimensions of the data.
3. Train a logistic regression model on the original and SVD-reduced data.
4. Measure and compare the training time and accuracy of the model with varying SVD components.
5. Plot the results and analyze how SVD impacts the performance and efficiency of the model.

***
Your tasks include:
1. Implement SVD algorithm. You are not allowed to directly use SVD implemented by other packages, but you may use functions in NumPy. (Part 2)
2. Explore the accuracy and time performance from different numbers of SVD components. (Part 4)
3. Visualize the accuracy, time performance and top 5 singular vectors in the dataset, analyze and explain which number of SVD component looks best to you? (Part 4,5&6) Hint: singular vectors should be reshaped to 28x28 images for visualization.
***
**Note that you may not import any other function or package.** Let's get started!


## Part 1: Load the MNIST dataset and preprocess the data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score, classification_report

# Load MNIST dataset
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target

# Normalize the data
X = X / 255.0

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Loading MNIST dataset...


C:\Users\kdesr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## Part 2: Implement SVD for Dimensionality Reduction

In [3]:
def apply_svd_custom(X_train, X_test, n_components):
    U, Sigma, V = svd(X_train)
    X_train_reduced = np.dot(U[:, :n_components], np.dot(Sigma[:n_components, :n_components], V[:n_components, :]))
    
    U, Sigma, V = svd(X_test)
    X_test_reduced = np.dot(U[:, :n_components], np.dot(Sigma[:n_components, :n_components], V[:n_components, :]))

    return X_train_reduced, X_test_reduced
    
def svd(A):    
    AtA = np.dot(A.T, A)                                            # find A^T * A
    eigvals, V = np.linalg.eigh(AtA)                                # find eigenvalues and eigenvectors

    idx = np.argsort(eigvals)[::-1]                                 # sort in descending order
    eigvals = np.absolute(eigvals[idx])
    V = V[:, idx]

    singular_values = np.sqrt(eigvals)                              # compute singular values

    U = np.dot(A, V)                                                # construct a normalized U
    U_normalized = np.zeros(U.shape)
    for i in range(U.shape[1]):
        if np.linalg.norm(U[:, i]) == 0:
            U_normalized[:, i] = U[:, i]
        else:
            U_normalized[:, i] = U[:, i] / np.linalg.norm(U[:, i])

    Sigma = np.zeros(A.shape)                                       # construct sigma
    for i in range(len(singular_values)):
        Sigma[i, i] = singular_values[i]

    return U_normalized, Sigma, V                                   # return U, sigma, V

## Part 3: Train Logistic Regression and Measure Performance

In [4]:
# Function to train logistic regression and track training time
def train_logistic_regression(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=1000, solver='saga', random_state=42, multi_class='multinomial')
    
    # Measure training time
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy, training_time


## Part 4: Experiment with Different Levels of SVD

Now, apply SVD with varying numbers of components and observe how the dimensionality reduction impacts the model's performance. Record both the accuracy and training time for each number of components.


In [9]:
svd_components = [5, 10, 25, 50, 100, 250]  # You need to decide what number to search...

# Store the results
results = [[],[],[]]

print("Training models with different levels of SVD preprocessing...")
for n_components in svd_components:
    print(f"Applying custom SVD with {n_components} components...")
    
    # Apply SVD to the training and test sets
    X_train_svd, X_test_svd = apply_svd_custom(X_train, X_test, n_components)
    
    # Train the logistic regression model and get accuracy and training time
    accuracy, training_time = train_logistic_regression(X_train_svd, y_train, X_test_svd, y_test)
        
    print(f"SVD components: {n_components}, Accuracy: {accuracy:.4f}, Training time: {training_time:.4f} seconds")
    results[0].append(n_components)
    results[1].append(accuracy)
    results[2].append(training_time)

Training models with different levels of SVD preprocessing...
Applying custom SVD with 5 components...
SVD components: 5, Accuracy: 0.1143, Training time: 17.5576 seconds
Applying custom SVD with 10 components...
SVD components: 10, Accuracy: 0.0909, Training time: 21.1798 seconds
Applying custom SVD with 25 components...
SVD components: 25, Accuracy: 0.1195, Training time: 43.1279 seconds
Applying custom SVD with 50 components...
SVD components: 50, Accuracy: 0.1091, Training time: 109.2557 seconds
Applying custom SVD with 100 components...
SVD components: 100, Accuracy: 0.1119, Training time: 232.1222 seconds
Applying custom SVD with 250 components...
SVD components: 250, Accuracy: 0.1079, Training time: 326.5073 seconds


## Part 5: Visualize and Analyze the Results

Finally, plot the accuracy, training time as a function of the number of SVD components, and top 5 singular vectors. This will help you understand the trade-off between dimensionality reduction, accuracy, and model training time, and how SVD generally works. Hint: singular vectors should be reshaped to 28x28 images for visualization.


In [6]:
## Your implementation here...
## You may add necessary lines in Part 4 to access data for visualization

## Part 6: Analyze / Conclusion 

YOUR ANSWER: 